## Import Libraries

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from tqdm import tqdm

In [3]:
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig
from pytorch_lightning import Callback, LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers import LightningLoggerBase

In [25]:
from src import utils

In [4]:
import pyrootutils

In [5]:
root = pyrootutils.setup_root(
    search_from='/scratch/tm3647/MLH/ConVIRT-private/',
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

## Load Hydra Config

In [6]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path="../../configs")
cfg = compose(config_name="pretraining")
#print(OmegaConf.to_yaml(cfg))

/ext3/fedml-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  after removing the cwd from sys.path.


In [7]:
cfg

{'task_name': 'convirt_pretrain', 'train': True, 'test': False, 'ckpt_path': None, 'seed': 42, 'datamodule': {'_target_': 'src.datamodules.mimiccxr_datamodule.MIMICCXRDataModule', 'batch_size': 32, 'num_workers': 4, 'pin_memory': False, 'text_model_name': 'emilyalsentzer/Bio_ClinicalBERT', 'mimic_cxr_dataset_file': '/scratch/tm3647/public/mimic_image_text_df.pkl'}, 'model': {'_target_': 'src.models.convirt_module.ConVIRTLitModule', 'optimizer': {'_target_': 'torch.optim.Adam', '_partial_': True, 'lr': 0.0001, 'weight_decay': 1e-06}, 'scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau', '_partial_': True, 'mode': 'min', 'factor': 0.5, 'patience': 5}, 'net': {'_target_': 'src.models.convirt_model.ConVIRT', 'image_model': {'_target_': 'src.models.image_encoder.ImageEncoder', 'name': 'resnet50', 'weights': 'IMAGENET1K_V2'}, 'text_model': {'_target_': 'src.models.text_encoder.TextEncoder', 'name': 'emilyalsentzer/Bio_ClinicalBERT', 'proj_dim_size': 512}, 'proj_dim_size': 5

## Load all Lightning Modules

In [8]:
datamodule: LightningDataModule = hydra.utils.instantiate(cfg.datamodule)

In [9]:
model: LightningModule = hydra.utils.instantiate(cfg.model)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
callbacks: List[Callback] = utils.instantiate_callbacks(cfg.get("callbacks"))

InterpolationResolutionError: ValueError raised while resolving interpolation: HydraConfig was not set

## Testing Data Module

In [10]:
datamodule.setup()

In [11]:
len(datamodule.train_dataloader()) * 32

342624

In [12]:
for batch in tqdm(datamodule.train_dataloader()):
    break

  0%|          | 0/10707 [00:09<?, ?it/s]


In [14]:
batch['image'].shape, batch['tokenized_data']['input_ids'].shape

(torch.Size([32, 3, 224, 224]), torch.Size([32, 128]))

In [31]:
len(datamodule.train_dataset) // 32

10706

In [33]:
len(datamodule.train_dataloader()) + len(datamodule.val_dataloader())

11271

In [35]:
11835 - 11271

564

In [34]:
len(datamodule.val_dataloader())

564

## Testing Model

In [16]:
model.net.text_model.model(**batch['tokenized_data'])

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.6133, -0.0309,  0.3905,  ..., -0.6321,  0.2037,  0.0981],
         [ 0.5240, -0.3419,  0.2443,  ...,  0.4413,  0.2786,  0.1616],
         [ 0.1511, -0.1453,  0.0757,  ...,  0.1246, -0.0336,  0.2544],
         ...,
         [ 0.0296, -0.1480,  0.0816,  ..., -0.0371, -0.0915,  0.1217],
         [ 0.1008, -0.2883,  0.1593,  ...,  0.0227, -0.1234,  0.0063],
         [ 0.2975, -0.0866,  0.1263,  ...,  0.3997, -0.0288,  0.1846]],

        [[ 0.3532, -0.1913,  0.1218,  ..., -0.5097,  0.2584, -0.2805],
         [-0.5346, -0.2125,  0.5341,  ...,  0.2782,  0.5559, -0.0804],
         [-0.1483,  0.0286,  0.7333,  ...,  0.0919,  0.4943,  0.0232],
         ...,
         [-0.1300, -0.2196,  0.0907,  ..., -0.3230, -0.1729,  0.0380],
         [ 0.0787, -0.3201,  0.3079,  ..., -0.1155,  0.1818,  0.0199],
         [-0.1882, -0.1797,  0.1351,  ..., -0.4061,  0.2764,  0.0084]],

        [[ 0.4161,  0.0030,  0.3831,  ..., -0.3789,  

In [18]:
v, u = model(batch)

In [19]:
v.shape

torch.Size([32, 512])

In [20]:
u.shape

torch.Size([32, 512])

## Testing Callbacks (Validating Config)

In [23]:
cfg.callbacks

{'model_checkpoint': {'_target_': 'pytorch_lightning.callbacks.ModelCheckpoint', 'dirpath': '${paths.output_dir}/checkpoints', 'filename': 'epoch_{epoch:03d}', 'monitor': 'val/loss', 'verbose': False, 'save_last': True, 'save_top_k': 1, 'mode': 'min', 'auto_insert_metric_name': False, 'save_weights_only': False, 'every_n_train_steps': None, 'train_time_interval': None, 'every_n_epochs': None, 'save_on_train_epoch_end': None}, 'model_summary': {'_target_': 'pytorch_lightning.callbacks.RichModelSummary', 'max_depth': -1}, 'rich_progress_bar': {'_target_': 'pytorch_lightning.callbacks.RichProgressBar'}}

## Testing Paths

## Testing Trainer Fit

## ....